<a href="https://colab.research.google.com/github/takzen/ai-engineering-handbook/blob/main/68_RLHF_PPO_ChatGPT_Alignment.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 🤖 RLHF & PPO: Jak powstał ChatGPT?

Trenowanie asystenta AI składa się z 3 faz:
1.  **Pre-training:** Model czyta internet i uczy się mówić (GPT-3).
2.  **SFT (Supervised Fine-Tuning):** Model uczy się formatu pytań i odpowiedzi.
3.  **RLHF (PPO):** Model uczy się, które odpowiedzi ludzie lubią bardziej.

**Problem z RL w tekstach:**
Jeśli nagrodzimy model tylko za "pozytywny wydźwięk", to model zacznie w kółko powtarzać słowo "Miłość Miłość Miłość", bo to daje max punktów. Traci zdolność mówienia po polsku.

**Rozwiązanie PPO (Proximal Policy Optimization):**
Mamy dwa modele:
1.  **Policy Model (Uczeń):** Ten, którego trenujemy.
2.  **Reference Model (Nauczyciel):** Kopia modelu z fazy 2 (zamrożona).

Liczymy **KL Divergence (Karę za odmienność)**. Jeśli Uczeń zaczyna gadać bzdury (zbyt różni się od Nauczyciela), dostaje karę, nawet jeśli nagroda za treść była wysoka.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import copy

# Konfiguracja
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LR = 3e-4
CLIP_EPS = 0.2  # Najważniejszy parametr PPO (o ile możemy zmienić politykę?)

print(f"Urządzenie: {DEVICE}")

Urządzenie: cuda


## Model Actor-Critic

W PPO potrzebujemy dwóch głów:
1.  **Actor:** Wybiera akcję (słowo). Zwraca prawdopodobieństwo.
2.  **Critic:** Ocenia stan. Mówi: "Jestem w dobrej sytuacji, spodziewam się dużej nagrody".

Dla uproszczenia zrobimy prostą sieć, która gra w grę liczbową (zamiast generować tekst, generuje liczby, które mają być bliskie celu).

In [2]:
class ActorCritic(nn.Module):
    def __init__(self, input_dim, action_dim):
        super().__init__()
        # Wspólny pień (Backbone)
        self.shared = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh()
        )
        
        # Głowa Aktora (Policy) - zwraca logity akcji
        self.actor = nn.Linear(64, action_dim)
        
        # Głowa Krytyka (Value) - zwraca jedną liczbę (ocenę sytuacji)
        self.critic = nn.Linear(64, 1)

    def forward(self, x):
        features = self.shared(x)
        action_logits = self.actor(features)
        state_value = self.critic(features)
        return action_logits, state_value

# Inicjalizacja
# Input: Stan gry (wektor 4 liczb)
# Action: 2 możliwe ruchy
model = ActorCritic(4, 2).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LR)

# Reference Model (Kopia, której nie trenujemy!)
ref_model = copy.deepcopy(model)
ref_model.eval() # Zamrażamy

print("Modele gotowe. Mamy Ucznia i Nauczyciela (Reference).")

Modele gotowe. Mamy Ucznia i Nauczyciela (Reference).


## Matematyka PPO (Clipped Loss)

To jest serce algorytmu. Wzór wygląda strasznie, ale idea jest prosta:
$$ Ratio = \frac{P_{nowe}}{P_{stare}} $$

Jeśli `Ratio` jest bliskie 1, to znaczy, że model zmienił się niewiele (bezpiecznie).
Jeśli `Ratio` jest np. 2.0 (model nagle 2x chętniej wybiera akcję), PPO ucina (clip) ten zysk, żeby model nie "wybuchł" od zbyt gwałtownej zmiany.

$$ Loss = - \min(Ratio \cdot A, \text{clip}(Ratio, 1-\epsilon, 1+\epsilon) \cdot A) $$
*   $A$ to Advantage (Zaleta) - czy akcja była lepsza niż oczekiwano?

In [3]:
def ppo_loss(old_log_probs, new_log_probs, advantages, returns, values):
    # 1. Obliczamy Ratio (r_t)
    # exp(new - old) to matematycznie to samo co (new_prob / old_prob)
    ratio = (new_log_probs - old_log_probs).exp()
    
    # 2. Surrogate Loss 1 (Zwykły)
    surr1 = ratio * advantages
    
    # 3. Surrogate Loss 2 (Przycięty - Clipped)
    # To jest hamulec bezpieczeństwa PPO!
    surr2 = torch.clamp(ratio, 1.0 - CLIP_EPS, 1.0 + CLIP_EPS) * advantages
    
    # 4. Loss Policy (Actor) - bierzemy minimum (pesymistyczne podejście)
    policy_loss = -torch.min(surr1, surr2).mean()
    
    # 5. Loss Value (Critic) - MSE między przewidywaniem a wynikiem
    value_loss = F.mse_loss(values.flatten(), returns)
    
    # Suma
    total_loss = policy_loss + 0.5 * value_loss
    return total_loss

print("Funkcja kosztu PPO zdefiniowana.")

Funkcja kosztu PPO zdefiniowana.


## Symulacja Treningu (Z karą KL)

Zasymulujemy jeden krok treningowy RLHF.
1.  **Rollout:** Model generuje akcję.
2.  **Reward:** Dostaje nagrodę od środowiska.
3.  **KL Penalty:** Sprawdzamy, jak bardzo ta akcja różni się od tego, co zrobiłby Reference Model.

In [5]:
# Symulacja danych (Batch)
states = torch.randn(10, 4).to(DEVICE) # 10 sytuacji
actions = torch.randint(0, 2, (10,)).to(DEVICE) # 10 podjętych decyzji
rewards = torch.tensor([1.0] * 5 + [-1.0] * 5).to(DEVICE) # Nagrody (5 dobrych, 5 złych)

# --- KROK 1: Obliczamy co myślą oba modele ---
# Nowy model (ten co się uczy)
logits, values = model(states)
new_log_probs = F.log_softmax(logits, dim=1)
# Wybieramy prawdop. tylko dla akcji, które faktycznie podjęliśmy
new_log_probs_actions = new_log_probs.gather(1, actions.unsqueeze(1)).flatten()

# Stary model (Reference - zamrożony)
with torch.no_grad():
    ref_logits, _ = ref_model(states)
    ref_log_probs = F.log_softmax(ref_logits, dim=1)
    ref_log_probs_actions = ref_log_probs.gather(1, actions.unsqueeze(1)).flatten()

# --- KROK 2: KL Divergence (Kara) ---
# Wzór: log(P_new) - log(P_ref)
kl_div = new_log_probs_actions - ref_log_probs_actions

# Odejmujemy KL od nagrody!
beta_kl = 0.1 # Siła kary
adjusted_rewards = rewards - (beta_kl * kl_div)

# --- POPRAWKA: DETACH ---
# Odpinamy nagrody od grafu. Traktujemy je teraz jako stałe liczby (target).
adjusted_rewards = adjusted_rewards.detach()

print("--- WPŁYW REFERENCE MODEL ---")
print(f"Oryginalna nagroda: {rewards[0].item()}")
print(f"Kara KL (odchylenie): {kl_div[0].item():.4f}")
print(f"Nagroda po korekcie: {adjusted_rewards[0].item():.4f}")

# --- KROK 3: PPO Update ---
# Uproszczone Advantage
advantages = adjusted_rewards 

# Zapisujemy "stare" prawdopodobieństwa (też odpięte, bo to kopia z przeszłości)
old_log_probs_actions = new_log_probs_actions.detach()

# Symulacja kilku epok PPO na tym samym batchu
for i in range(5):
    optimizer.zero_grad()
    
    # Recalculate (bo model się zmienia w pętli - budujemy nowy graf w każdej iteracji)
    logits, values = model(states)
    curr_log_probs = F.log_softmax(logits, dim=1).gather(1, actions.unsqueeze(1)).flatten()
    
    # Teraz adjusted_rewards jest bezpieczne (detached)
    loss = ppo_loss(old_log_probs_actions, curr_log_probs, advantages, adjusted_rewards, values)
    
    loss.backward()
    optimizer.step()
    print(f"PPO Step {i}: Loss={loss.item():.4f}")

--- WPŁYW REFERENCE MODEL ---
Oryginalna nagroda: 1.0
Kara KL (odchylenie): -0.0190
Nagroda po korekcie: 1.0019
PPO Step 0: Loss=0.4167
PPO Step 1: Loss=0.4132
PPO Step 2: Loss=0.4071
PPO Step 3: Loss=0.3999
PPO Step 4: Loss=0.3922


## 🧠 Podsumowanie: Alignment Tax

To, co zrobiliśmy (dodanie kary `beta_kl * kl_div`), nazywa się w branży **Alignment Tax (Podatek od dopasowania)**.

Model staje się "grzeczniejszy" i bardziej zgodny z instrukcjami człowieka (RLHF), ale przez to, że musi trzymać się blisko modelu bazowego (Reference), traci trochę ze swojej kreatywności i "inteligencji".

**Kluczowe wnioski:**
1.  **PPO Clip:** Zapobiega gwałtownym zmianom modelu (stabilność).
2.  **Reference Model:** Działa jak kotwica. Nie pozwala modelowi zapomnieć języka polskiego, gdy uczy się być miłym asystentem.